In [1]:
%matplotlib inline
import numpy as np
import paddle
import paddle.fluid as fluid
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os


z_dim = 100
batch_size = 128
step_per_epoch = 60000 / batch_size

Generator & Discriminator

In [2]:
def generator(z, name="G"):
    with fluid.unique_name.guard(name + "/"):
        y = z
        
        y = fluid.layers.fc(y, size=1024, act='tanh')
        y = fluid.layers.fc(y, size=128 * 7 * 7)
        y = fluid.layers.batch_norm(y, act='tanh')
        y = fluid.layers.reshape(y, shape=(-1, 128, 7, 7))

        y = fluid.layers.image_resize(y, scale=2)
        y = fluid.layers.conv2d(y, num_filters=64, filter_size=5, padding=2, act='tanh')

        y = fluid.layers.image_resize(y, scale=2)
        y = fluid.layers.conv2d(y, num_filters=1, filter_size=5, padding=2, act='tanh')

    return y
        

In [3]:
def discriminator(image, name="D"):
    with fluid.unique_name.guard(name+'_'):
        conv1 = fluid.layers.conv2d(input=image, num_filters=32,
                                    filter_size=6, stride=2,
                                    padding=2)
        conv1_act = fluid.layers.leaky_relu(conv1)
        
        conv2 = fluid.layers.conv2d(conv1_act, num_filters=64, 
                                    filter_size=6, stride=2,
                                    padding=2)
        conv2 = fluid.layers.batch_norm(conv2)
        conv2_act = fluid.layers.leaky_relu(conv2)
        
        fc1 = fluid.layers.reshape(conv2_act, shape=(-1,64*7*7))
        fc1 = fluid.layers.fc(fc1, size=512)
        fc1_bn = fluid.layers.batch_norm(fc1)
#         fc1_act = fluid.layers.leaky_relu(fc1_bn)
        
        #有没有sigmoid的act???
#         fc2 = fluid.layers.fc(fc1_act, size=1, act='sigmoid')
        fc2 = fluid.layers.fc(fc1_bn, size=1)
        print("fc2",fc2)
        return fc2

Training

In [4]:
def get_params(program, prefix):
    all_params = program.global_block().all_parameters()
    return [t.name for t in all_params if t.name.startswith(prefix)]

In [5]:
#优化generator
G_program = fluid.Program()
with fluid.program_guard(G_program):
    z = fluid.layers.data(name='z', shape=[z_dim,1,1])
    # 用生成器G生成样本图片
    G_sample = generator(z)
    infer_program = G_program.clone(for_test=True)
    # 用判别器D判别生成的样本
    D_fake = discriminator(G_sample)
    
#     ones = fluid.layers.fill_constant_batch_size_like(z, shape=[-1, 1], dtype='float32', value=1)
    # G损失
    # G Least square cost
    G_loss = fluid.layers.reduce_mean(fluid.layers.square(D_fake-1))/2.
    # 获取G的参数
    G_params = get_params(G_program, "G")
    
    # 使用Adam优化器
    G_optimizer = fluid.optimizer.RMSPropOptimizer(learning_rate=0.001)
    # 训练G
    G_optimizer.minimize(G_loss,parameter_list = G_params)
    print(G_params)

('fc2', name: "D_D_fc_1.tmp_1"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 1
    }
    lod_level: 0
  }
}
persistable: false
)
[u'G/G/fc_0.w_0', u'G/G/fc_0.b_0', u'G/G/fc_1.w_0', u'G/G/fc_1.b_0', u'G/G/batch_norm_0.w_0', u'G/G/batch_norm_0.b_0', u'G/G/batch_norm_0.w_1', u'G/G/batch_norm_0.w_2', u'G/G/conv2d_0.w_0', u'G/G/conv2d_0.b_0', u'G/G/conv2d_1.w_0', u'G/G/conv2d_1.b_0']


In [6]:
# 优化discriminator
D_program = fluid.Program()
with fluid.program_guard(D_program):
    z = fluid.layers.data(name='z', shape=[z_dim,1,1])
    # 用生成器G生成样本图片
    G_sample = generator(z)
    real = fluid.layers.data(name='img', shape=[1, 28, 28])
    # 用判别器D判别真实的样本
    D_real = discriminator(real)
    # 用判别器D判别生成的样本
    D_fake = discriminator(G_sample)
    # D损失
    print("D_real",D_real)
    print("D_fake",D_fake)
    # D Least square cost
    D_loss = fluid.layers.reduce_mean(fluid.layers.square(D_real-1.)+fluid.layers.square(D_fake))/2.
    print("D_loss",D_loss)
    # 获取D的参数列表
    D_params = get_params(D_program, "D")
    # 使用Adam优化
    D_optimizer = fluid.optimizer.RMSPropOptimizer(learning_rate=0.001)
    D_optimizer.minimize(D_loss, parameter_list = D_params)
    print(D_params)

('fc2', name: "D_D_fc_1.tmp_1"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 1
    }
    lod_level: 0
  }
}
persistable: false
)
('fc2', name: "D_D_fc_1.tmp_1"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 1
    }
    lod_level: 0
  }
}
persistable: false
)
('D_real', name: "D_D_fc_1.tmp_1"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 1
    }
    lod_level: 0
  }
}
persistable: false
)
('D_fake', name: "D_D_fc_1.tmp_1"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 1
    }
    lod_level: 0
  }
}
persistable: false
)
('D_loss', name: "tmp_8"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: 1
    }
    lod_level: 0
  }
}
)
[u'D_D_conv2d_0.w_0', u'D_D_conv2d_0.b_0', u'D_D_conv2d_1.w_0', u'D_D_conv2d_1.b_0', u'D_D_batch_norm_0.

In [7]:
#读入数据，只载入训练集
# train_reader = paddle.batch(paddle.reader.shuffle(paddle.dataset.mnist.train(),
#                                                     buf_size = 1024),
#                                                     batch_size = batch_size)
# #Executor
# exe = fluid.Executor(fluid.CPUPlace())
# exe.run(program=fluid.default_startup_program())

# #Inference
# Infer_program = fluid.Program()
# with fluid.program_guard(main_program = Infer_program):   
#     z = fluid.layers.data(name='z', shape=[-1,z_dim], dtype='float32')
#     G_sample = generator(z)
    
# it = 0
# for _ in range(1000):
#     for data in train_reader():
#         it += 1
#         # 获取训练集图像
#         X_mb = [data[i][0] for i in range(batch_size)]
#         Z_noise = np.random.uniform(-1., 1., size=[batch_size,z_dim])
        
#         D_feeding={"img" : np.array(X_mb).astype('float32'),
#                     "z" : np.array(Z_noise).astype('float32')}
        
#         G_feeding={"z" : np.array(Z_noise).astype('float32')}
        
# #         D_loss_curr = exe.run(feed = D_feeding, 
# #                       program = D_program, 
# #                       fetch_list = [D_loss])

#         G_loss_curr = exe.run(feed = G_feeding, 
#                               program = G_program, 
#                               fetch_list = [G_loss])
        
# #         if it % 1000 == 0:
# #             print(str(it) + ' | ' 
# #                   + str (D_loss_curr[0][0]) + ' | ' 
# #                   + str (G_loss_curr[0][0]))
            
#         if it % 10000 == 0:
#             #显示模型生成结果
#             Z_noise = np.random.uniform(-1., 1., size=[batch_size,z_dim])
#             G_feeding={"z" : np.array(Z_noise).astype('float32')}
#             samples = exe.run(feed = G_feeding,
#                               program = Infer_program,
#                               fetch_list = [G_sample])
#             plt.subplot(241)
#             plt.imshow(np.reshape(X_mb[0], [28, 28]), 
#                        cmap=plt.cm.BuPu_r)
#             for i in range(7):
#                 plt.subplot(242 + i)
#                 plt.imshow(np.reshape(samples[0][i], [28,28]), 
#                            cmap=plt.cm.BuPu_r)
#             plt.show()

In [8]:
def show_image_grid(images, epoch=None): # images.shape = (64, 1, 28, 28)
    fig = plt.figure(figsize=(5, 5))
    fig.suptitle("Epoch {}".format(epoch))
    gs = plt.GridSpec(8, 8)
    gs.update(wspace=0.05, hspace=0.05)

    for i, image in enumerate(images[:64]):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
#         plt.imshow(image[0], cmap='Greys_r')
        plt.imshow(image[0], cmap='Greys_r')
    plt.show()

In [9]:
batch_size = 128
step_per_epoch = 60000 / batch_size

# 噪声生成
def z_reader():
    while True:
        yield np.random.normal(0.0, 1.0, (z_dim, 1, 1)).astype('float32')

In [10]:
# MNIST数据集，不使用label
def mnist_reader(reader):
    def r():
        for img, label in reader():
            yield img.reshape(1, 28, 28)
    return r

mnist_generator = paddle.batch(
    paddle.reader.shuffle(mnist_reader(paddle.dataset.mnist.train()), 1024), batch_size=batch_size)
z_generator = paddle.batch(z_reader, batch_size=batch_size)()

In [11]:
place = fluid.CUDAPlace(0) if fluid.core.is_compiled_with_cuda() else fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(program=fluid.default_startup_program())

[]

In [12]:
np.random.seed(0)
test_z = np.array(next(z_generator))
#print("test_z",test_z) 

In [ ]:
for epoch in range(100):
    epoch_d_loss = []
    epoch_g_loss = []
    
    for i, real_image in enumerate(mnist_generator()):
        step = epoch * step_per_epoch + i
        
        # 训练D识别真实图片 
        r_d = exe.run(D_program, fetch_list=[D_loss], feed={
            'img': np.array(real_image),
            'z': np.array(next(z_generator))
        })
        epoch_d_loss.append(np.mean(r_d))
        
        ## 训练G生成符合D标准的“真实”图片
        r_g = exe.run(G_program, fetch_list=[G_loss], feed={
            'z': np.array(next(z_generator))
        })
        epoch_g_loss.append(np.mean(r_g))
        
        if i % 50 == 0:
            print("Epoch {} batch {} d {} g {}".format(
                epoch, i, np.mean(epoch_d_loss), np.mean(epoch_g_loss)
            ))
        
    # 测试
    r_i = exe.run(infer_program, fetch_list=[G_sample], feed={
        'z': test_z
    })
    show_image_grid(r_i[0], epoch)

Epoch 0 batch 0 d 2.36631250381 g 184.83972168
Epoch 0 batch 50 d 14.2643966675 g 6.03646087646
Epoch 0 batch 100 d 7.37970256805 g 3.13961005211
Epoch 0 batch 150 d 5.05142974854 g 2.15960288048
Epoch 0 batch 200 d 3.88256311417 g 1.66660952568
Epoch 0 batch 250 d 3.17431688309 g 1.36731469631
Epoch 0 batch 300 d 2.70140838623 g 1.16741204262
